In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

Generator

In [3]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 25),    # transforms noise (input_dim random inputs) into structure
            nn.ReLU(),      
            nn.Linear(25, 50),         # transforms the structure into a larger structure
            nn.ReLU(),
            nn.Linear(50, 25),          # start condensing the learned patters
            nn.ReLU(),
            nn.Linear(25, output_dim),
            nn.Tanh()                   # ensures all values fall within a range of -1 to 1
        )

    def forward(self, noise):
        return self.layers(noise)
    
generator = Generator(input_dim=20, output_dim=12)


Discriminator

In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),    
            nn.LeakyReLU(0.2),           # to prevent dying neurons
            nn.Linear(64, 64),          
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),            # outputting a single value
            nn.Sigmoid()                 # sigmoid to squish the output between 0 and 1 (real or fake)
        )
        
    def forward(self, x):       # accepts either real or fake data,
        return self.net(x)      # and returns the probability of data being real
    

discriminator = Discriminator(input_dim=12)

Training

In [28]:
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

features_scaled_path = r'../Planetary Data/features_data.csv'
features_scaled = pd.read_csv(features_scaled_path, sep='\t')

features_tensor = torch.tensor(features_scaled.values).float()
print(features_scaled.shape) # should be (575, 12)


dataset = TensorDataset(features_tensor)
batch_size = 64     # number of samples in each batch
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

(575, 12)


Training Loop